In [2]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 09
### Team Member Names: Aryan Singh, Jack Smith, and Samyak Jain
### Team Strategy Chosen: Market Meet (Market Beat, Market Meet, Risk-Free)

In [15]:
tickers_df = pd.DataFrame({
     'Tickers': ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'BAC', 'C', 'WFC', 'GS',
                 'XOM', 'CVX', 'SLB', 'COP', 'AMZN', 'TSLA', 'HD', 'NKE',
                 'JNJ', 'PFE', 'UNH', 'ABBV', 'KO', 'PEP', 'WMT', 'COST',
                 'NEE', 'DUK', 'SO', 'D', 'AMT', 'PLD', 'EQIX', 'SPG',
                 'FCX', 'NEM', 'LIN', 'APD', 'TD.TO']
 })

tickers_df

,Tickers
0,AAPL
1,MSFT
2,GOOGL
3,JPM
4,BAC
5,C
6,WFC
7,GS
8,XOM
9,CVX


In [16]:
# what the benchmark weighting on different sectors should be

bench_weights = {
    'Financials': 0.229,
    'Information Technology': 0.198,
    'Industrials':0.1115,
    'Energy': 0.0875,
    'Consumer Discretionary': 0.00705,
    'Communication Services': 0.0635,
    'Materials': 0.0625,
    'Consumer Staples': 0.0515,
    #'Health Care': 0.0495,
    'Healthcare': 0.0495,  # Handle both naming conventions
    'Utilities': 0.019,
    'Real Estate': 0.0105}
    
sector_mapping = {
    'Financial Services': 'Financials',
    'Technology': 'Technology',
    'Industrials': 'Industrials',
    'Energy': 'Energy',
    'Consumer Cyclical': 'Consumer Discretionary',
    'Communication Services': 'Communication Services',
    'Basic Materials': 'Materials',
    'Consumer Defensive': 'Consumer Staples',
    'Healthcare': 'Health Care',
    'Utilities': 'Utilities',
    'Real Estate': 'Real Estate'
}


In [17]:
Initial_capital = 1_000_000
fee_flat_usd = 2.15
fee_per_share_usd = 0.001
stocks_per_sector= 2

tickers_list = []
sectors_list = []
prices_list = []
currencies_list = []
market_caps_list = []

for ticker in tickers_df['Tickers']:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        yf_sector = info.get('sector', 'Unknown')
        sector = sector_mapping.get(yf_sector)

        if sector == 'Unknown':
            continue

        price = stock.history(period='1d')['Close'].iloc[-1] #current price

        currency = info.get('currency', 'USD')
        market_cap = info.get('marketCap', 0)

        tickers_list.append(ticker)
        sectors_list.append(sector)
        prices_list.append(price)
        currencies_list.append(currency)
        market_caps_list.append(market_cap)

    except:
        continue

df = pd.DataFrame({
    'Ticker':tickers_list,
    'Sector':sectors_list,
    'Price': prices_list,
    'Currency': currencies_list,
    'Market Cap': market_caps_list })

print(df.to_markdown())
print(f"Sector By Size: {df.groupby('Sector').size()}")

HTTP Error 404: 
$ABAASDS: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


|    | Ticker   | Sector                 |   Price | Currency   |    Market Cap |
|---:|:---------|:-----------------------|--------:|:-----------|--------------:|
|  0 | AAPL     | Technology             | 267.66  | USD        | 3972178640896 |
|  1 | MSFT     | Technology             | 491.34  | USD        | 3651988881408 |
|  2 | GOOGL    | Communication Services | 284.41  | USD        | 3444808613888 |
|  3 | JPM      | Financials             | 299.27  | USD        |  822918774784 |
|  4 | BAC      | Financials             |  51.715 | USD        |  383050285056 |
|  5 | C        | Financials             |  98.32  | USD        |  180997079040 |
|  6 | WFC      | Financials             |  84.09  | USD        |  269345341440 |
|  7 | GS       | Financials             | 780.08  | USD        |  236146671616 |
|  8 | XOM      | Energy                 | 118.04  | USD        |  503233708032 |
|  9 | CVX      | Energy                 | 152.39  | USD        |  307038781440 |
| 10 | SLB      

In [18]:
# calculating annual volatility

end_date = datetime.now()
start_date = datetime(end_date.year - 1, end_date.month, end_date.day) # 1 year ago

trading_days_per_year = 251 # around 251 trading days per year.
min_days_required = 60 # uses traded stocks

volatilities = []

for ticker in df['Ticker']:

    try:
        
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        if hist is None or hist.empty or len(hist) < min_days_required:
            volatilities.append(np.nan)
            continue
            
        returns = hist['Close'].pct_change().dropna()
            
        if returns.empty:
            volatilities.append(np.nan)
            continue
                
        annual_vol = returns.std() * np.sqrt(trading_days_per_year)
        volatilities.append(annual_vol)
    
    except Exception as e:
        volatilities.append(np.nan)
        continue
    
df['Volatility'] = volatilities

df = df.dropna(subset=['Volatility']).reset_index(drop=True) # dropping all stocks with nan volatility.

df


,Ticker,Sector,Price,Currency,Market Cap,Volatility
0,AAPL,Technology,267.660004,USD,3972178640896,0.325338
1,MSFT,Technology,491.339996,USD,3651988881408,0.243140
2,GOOGL,Communication Services,284.410004,USD,3444808613888,0.328676
3,JPM,Financials,299.269989,USD,822918774784,0.241966
4,BAC,Financials,51.715000,USD,383050285056,0.268943
5,C,Financials,98.320000,USD,180997079040,0.311487
6,WFC,Financials,84.089996,USD,269345341440,0.291824
7,GS,Financials,780.080017,USD,236146671616,0.310417
8,XOM,Energy,118.040001,USD,503233708032,0.233521
9,CVX,Energy,152.389999,USD,307038781440,0.246058


In [11]:
print("Before filtering:", len(df))
print(df.groupby("Sector").size())

Before filtering: 37
Sector
Communication Services    1
Consumer Discretionary    4
Consumer Staples          4
Energy                    4
Financials                6
Health Care               4
Materials                 4
Real Estate               4
Technology                2
Utilities                 4
dtype: int64


In [12]:
# I have already created the dataframe with each stock, its secor, its currency, market_cap, and annual volatility
#use this dataframe throughout the rest of the code. It is okay to manupulate the dataframe to remove stocks
#in which are not deemed fit to invest in.

# get rid of stocks with high volatility. i.e. get rid of the stocks with the highest std within each sector. I'm thinking maybe the top 25%

cutoff = 0.75

filtered_rows = []
seen_sectors = []

for i in range(len(df)):
    sector_name = df.iloc[i]['Sector']
    if sector_name not in seen_sectors:
        seen_sectors.append(sector_name)

for sector in seen_sectors:
    
    sector_df = df[df['Sector'] == sector].copy()
    if len(sector_df) < 4:
        filtered_rows.append(sector_df)
        continue
    
    sector_df = sector_df.sort_values(by='Volatility').reset_index(drop=True)
        
    keep_count = int(len(sector_df) * cutoff)
    if keep_count < 1:
        keep_count = 1
    
    sector_kept = sector_df.iloc[:keep_count]
    filtered_rows.append(sector_kept)

df = pd.concat(filtered_rows, ignore_index=True)

df


,Ticker,Sector,Price,Currency,Market Cap,Volatility
0,AAPL,Technology,267.869995,USD,3975295008768,0.325993
1,MSFT,Technology,491.470001,USD,3653178228736,0.243038
2,GOOGL,Communication Services,284.739990,USD,3448805523456,0.328672
3,TD.TO,Financials,114.820000,CAD,197774917632,0.167353
4,JPM,Financials,299.394989,USD,823262511104,0.242407
5,BAC,Financials,51.785000,USD,383568773120,0.268943
6,WFC,Financials,84.261002,USD,269925154816,0.291863
7,XOM,Energy,118.125000,USD,503596056576,0.233081
8,CVX,Energy,152.499893,USD,307260194816,0.246045
9,COP,Energy,88.750000,USD,110843633664,0.331784


In [13]:
print("After filtering:", len(df))
print(df.groupby("Sector").size())

After filtering: 28
Sector
Communication Services    1
Consumer Discretionary    3
Consumer Staples          3
Energy                    3
Financials                4
Health Care               3
Materials                 3
Real Estate               3
Technology                2
Utilities                 3
dtype: int64


In [14]:
# create the benchmark. Basically, create a dataframe that consists of the closing returns of the benchmark within the past year(1 year because we are going to 
#track the correlation between the benchmark and the stocks back by one year later. The benchmark is just an equally weighted average of the
#TSX composite and the S&P500 returns.

end_date = datetime.now()
start_date = datetime(end_date.year - 1, end_date.month, end_date.day)

tsx_symbol = "^GSPTSE"
spx_symbol = "^GSPC"

tsx_ticker = yf.Ticker(tsx_symbol)
spx_ticker = yf.Ticker(spx_symbol)

tsx_hist = tsx_ticker.history(start=start_date, end=end_date)
spx_hist = spx_ticker.history(start=start_date, end=end_date)

# Maybe needed
# if tsx_hist is None or tsx_hist.empty:
#     print("Warning: TSX data missing.")

# if spx_hist is None or spx_hist.empty:
#     print("Warning: S&P 500 data missing.")

tsx_ret = tsx_hist['Close'].pct_change()
spx_ret = spx_hist['Close'].pct_change()

benchmark_df = pd.DataFrame()
benchmark_df['TSX Return'] = tsx_ret
benchmark_df['S&P 500 Return'] = spx_ret

benchmark_df = benchmark_df.dropna()

benchmark_df['Benchmark Return'] = (benchmark_df['TSX Return'] + benchmark_df['S&P 500 Return']) / 2
benchmark_df.index = benchmark_df.index.strftime('%Y-%m-%d')

benchmark_df


,TSX Return,S&P 500 Return,Benchmark Return
Date,,,
2024-11-19,0.001357,0.003964,0.002660
2024-11-20,0.001028,0.000022,0.000525
2024-11-21,0.014147,0.005340,0.009744
2024-11-22,0.002111,0.003468,0.002790
2024-11-25,-0.001332,0.003020,0.000844
...,...,...,...
2025-11-12,0.013756,0.000630,0.007193
2025-11-13,-0.018620,-0.016557,-0.017588
2025-11-14,0.002410,-0.000502,0.000954


In [36]:
## impliment a market_cap check. Basically get rid of any pennystocks as they don't tend to always follow the market very well.



In [71]:
# Pick the top 1-3 stocks in each sector with the best correlation to the benchmark over the past year.


In [72]:
# Create the weighting of the stocks baised on the benchmark weighting as define above. If there aren't any
#good stocks in a particular low-weighted sector within the benchmark, you can just re-distribute the weight
#into the higher weighted sectors and stocks. Also during this process you need to make sure that 
#the weightings follow the portfolio rules in terms of weightings and restrictions.



## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.